# Designe and deploye an end-to-end pipeline for product price prediction, incorporating model training and evaluation

### steps
1 Scrape data <br>
2 Upload to google sheet <br>
3 Visualize on DataStudio<br>
4 Connect with Dialogflow (AVG, MAX, MIN price)<br>
5 Make prediction model <br>

# Scrape Data from multiple Sites

In [103]:
from bs4 import BeautifulSoup
import requests
from selectorlib import Extractor
import math
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def purchase_valley_data(product_name, page_number=1):
    # use creds to create a client to interact with the Google Drive API
    creds = ServiceAccountCredentials.from_json_keyfile_name('test-bq-rc-demo-a56264e3ae52.json')
    client = gspread.authorize(creds)
    sheet = client.open("ScrapeDataSheet").sheet1
    
    global all_data
    all_data = []
    url = f"https://purchasevalley.com/?s={product_name}&product_cat=0&post_type=product"
    if page_number!=1:
        url = f"https://purchasevalley.com/page/{page_number}/?s={product_name}&product_cat=0&post_type=product"
    session = requests.Session()
    session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.1.2222.33 Safari/537.36"
    }
    response = session.get(url=url)
    soup = BeautifulSoup(response.text, 'html.parser')

    yaml_ = '''
    total_items:
        css: 'p.woocommerce-result-count'
        xpath: null
        multiple: false
        type: Text
    product_data:
        css: 'li.thunk-woo-product-list'
        xpath: null
        multiple: true
        type: Text
        children:
            product_name:
                css: 'h2.woocommerce-loop-product__title'
                xpath: null
                multiple: true
                type: Text
            product_link:
                css: 'h2.woocommerce-loop-product__title'
                xpath: null
                multiple: true
                type: Link
            single_price:
                css: 'bdi'
                xpath: null
                multiple: true
                type: Text
            old_price:
                css: 'del bdi'
                xpath: null
                multiple: true
                type: Text
            price:
                css: 'ins bdi'
                xpath: null
                multiple: true
                type: Text
            '''

    extractor = Extractor.from_yaml_string(yaml_)

    data = extractor.extract(response.text)
    all_data += data['product_data']
    # print(all_data)

    # Send data to Google Sheet code Here
    index = 2                               # add data from 2nd row | First Row = Header
    for itme in all_data:
        nested_item_list = list(itme.values())
        # print(nested_item_list)
        try:
            flatten_item_list = [ item for elem in nested_item_list for item in elem]
            flatten_item_list.append(product_name)
            sheet.insert_row(flatten_item_list, index)
        except:
            flatten_items = []
            flatten_items.append(nested_item_list[0][0])
            flatten_items.append(nested_item_list[1][0])
            flatten_items.append(nested_item_list[2][0])
            flatten_items.append(nested_item_list[3])
            flatten_items.append(nested_item_list[4])
            flatten_items.append(product_name)
            sheet.insert_row(flatten_items, index)
    
    print(data['total_items'])
    if page_number == 1:
        if data['total_items'].startswith("Showing all"):
            print("Total 1 page")
        else:
            pages = data['total_items'].split()
            total_item = int(pages[3])
            per_page_item = int(pages[1].split("–")[1])
            number_of_pages = math.ceil(total_item/per_page_item)
            print("Number of pages:",number_of_pages)
            for next_page in range(2,number_of_pages+1):
                purchase_valley_data(product_name=product_name, page_number=next_page)
    else:
        pass


In [64]:
from bs4 import BeautifulSoup
import requests
from selectorlib import Extractor
import math
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

def purchase_valley_data(product_name, page_number=1):
    # use creds to create a client to interact with the Google Drive API
    creds = ServiceAccountCredentials.from_json_keyfile_name('test-bq-rc-demo-a56264e3ae52.json')
    client = gspread.authorize(creds)
    sheet = client.open("ScrapeDataSheet").sheet1
    
    global all_data
    all_data = []
    url = f"https://purchasevalley.com/?s={product_name}&product_cat=0&post_type=product"
    if page_number!=1:
        url = f"https://purchasevalley.com/page/{page_number}/?s={product_name}&product_cat=0&post_type=product"
    session = requests.Session()
    session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.1.2222.33 Safari/537.36"
    }
    response = session.get(url=url)
    soup = BeautifulSoup(response.text, 'html.parser')

    yaml_ = '''
    total_items:
        css: 'p.woocommerce-result-count'
        xpath: null
        multiple: false
        type: Text
    product_data:
        css: 'li.thunk-woo-product-list'
        xpath: null
        multiple: true
        type: Text
        children:
            product_name:
                css: 'h2.woocommerce-loop-product__title'
                xpath: null
                multiple: true
                type: Text
            product_link:
                css: 'h2.woocommerce-loop-product__title'
                xpath: null
                multiple: true
                type: Link
            price:
                css: 'bdi'
                xpath: null
                multiple: true
                type: Text
            '''

    extractor = Extractor.from_yaml_string(yaml_)

    data = extractor.extract(response.text)
    all_data += data['product_data']
    # print(all_data)

    # Send data to Google Sheet code Here
    index = 2                               # add data from 2nd row | First Row = Header
    for itme in all_data:
        nested_item_list = list(itme.values())
        # print(nested_item_list)
        try:
            flatten_item_list = [ item for elem in nested_item_list for item in elem]
            flatten_item_list.insert(2, 'PurchaseValley')
            flatten_item_list.insert(3, product_name)
            flatten_item_list[4] = flatten_item_list[4][2:]     # Exclude Rs
            try:
                flatten_item_list[5] = flatten_item_list[5][2:] 
            except:
                flatten_item_list[5] = 0
            sheet.insert_row(flatten_item_list, index)

        # For Single Page Results
        except:
            flatten_items = []
            flatten_items.append(nested_item_list[0][0])
            flatten_items.append(nested_item_list[1][0])
            try:
                flatten_items.append(nested_item_list[2][0][2:])
            except:
                flatten_items.append(None)              # Items with no price
            flatten_items.insert(2, 'PurchaseValley')
            flatten_items.insert(3, product_name)
            sheet.insert_row(flatten_items, index)
    
    print(data['total_items'])
    if page_number == 1:
        if data['total_items'].startswith("Showing all"):
            print("Total 1 page")
        else:
            pages = data['total_items'].split()
            total_item = int(pages[3])
            per_page_item = int(pages[1].split("–")[1])
            number_of_pages = math.ceil(total_item/per_page_item)
            print("Number of pages:",number_of_pages)
            for next_page in range(2,number_of_pages+1):
                time.sleep(20)          # Quota exceeded Write requests per minute per user
                purchase_valley_data(product_name=product_name, page_number=next_page)
    else:
        pass


In [65]:

purchase_valley_data(product_name='kurta')


Showing 1–16 of 78 results
Number of pages: 5
Showing 17–32 of 78 results
Showing 33–48 of 78 results
Showing 49–64 of 78 results
Showing 65–78 of 78 results


In [66]:
purchase_valley_data(product_name='shirt')

Showing 1–16 of 125 results
Number of pages: 8
Showing 17–32 of 125 results
Showing 33–48 of 125 results
Showing 49–64 of 125 results
Showing 65–80 of 125 results
Showing 81–96 of 125 results
Showing 97–112 of 125 results
Showing 113–125 of 125 results


In [67]:
purchase_valley_data(product_name='belt')

Showing all 10 results
Total 1 page


# bagallery Scrapping

In [69]:
# bagallery.com
product_name = "shirt"
page_number = 1
search_link = f"https://bagallery.com/search?type=product&q={product_name}"
search_link

'https://bagallery.com/search?type=product&q=shirt'

In [ ]:
from bs4 import BeautifulSoup
import requests
from selectorlib import Extractor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

def bagallery_data(product_name, page_number=1):

    creds = ServiceAccountCredentials.from_json_keyfile_name('test-bq-rc-demo-a56264e3ae52.json')
    client = gspread.authorize(creds)
    sheet = client.open("ScrapeDataSheet").sheet1
    index = 2       # insert Data From 2nd row
    flatten_all_item_list = []

    while True:
        url = f'https://bagallery.com/search?page={page_number}&type=product&q={product_name}'
        session = requests.Session()
        response = session.get(url=url)

        yaml_ = '''
        product_data:
            css: 'div.grid-item'
            xpath: null
            multiple: true
            type: Text
            children:
                product_name:
                    css: 'a.product-title span'
                    xpath: null
                    multiple: true
                    type: Text
                product_link:
                    css: 'a.product-title span'
                    xpath: null
                    multiple: true
                    type: Link
                old_price:
                    css: 'span.old-price'
                    xpath: null
                    multiple: true
                    type: Text
                discounted_price:
                    css: 'span.special-price'
                    xpath: null
                    multiple: true
                    type: Text
                price:
                    css: 'div.price-regular span'
                    xpath: null
                    multiple: true
                    type: Text
                '''

        extractor = Extractor.from_yaml_string(yaml_)

        data = extractor.extract(response.text)

        page_number += 1
        try:
            for itme in data['product_data']:
                flatten_item_list = []
                flatten_item_list.append(itme['product_name'][0])
                flatten_item_list.append(itme['product_link'])
                flatten_item_list.append("BaGallery")
                flatten_item_list.append(product_name)
                try:
                    flatten_item_list.append(itme['price'][0])
                except:
                    flatten_item_list.append(itme['old_price'][0])
                    flatten_item_list.append(itme['discounted_price'][0])
                flatten_all_item_list.append(flatten_item_list)
        except:
            break       # If no data then Break
    
    for flatten_item_list in flatten_all_item_list:
        try:
            sheet.insert_row(flatten_item_list, index)
        except:
            time.sleep(20)

        # print("Page number:", page)
        # print(data['product_data'])


[{'product_name': ['Blue Men’s Kurta – 2'],
  'product_link': None,
  'old_price': ['₨ 900.00'],
  'price': ['₨ 225.00']},
 {'product_name': ['New York Pink Men’s Kurta'],
  'product_link': None,
  'old_price': ['₨ 900.00'],
  'price': ['₨ 225.00']},
 {'product_name': ['Paprika Men’s Kurta'],
  'product_link': None,
  'old_price': ['₨ 900.00'],
  'price': ['₨ 225.00']},
 {'product_name': ['Atoll Men’s Kurta'],
  'product_link': None,
  'old_price': ['₨ 900.00'],
  'price': ['₨ 225.00']},
 {'product_name': ['Sandstone Men’s Kurta – 2'],
  'product_link': None,
  'old_price': ['₨ 700.00'],
  'price': ['₨ 175.00']},
 {'product_name': ['Sandstone Men’s Kurta – 1'],
  'product_link': None,
  'old_price': ['₨ 750.00'],
  'price': ['₨ 188.00']},
 {'product_name': ['Hurricane Men’s Kurta'],
  'product_link': None,
  'old_price': ['₨ 750.00'],
  'price': ['₨ 188.00']},
 {'product_name': ['River Bed Men’s Kurta – 3 – Medium Size'],
  'product_link': None,
  'old_price': ['₨ 750.00'],
  'price': 

In [ ]:

bagallery_data(product_name='kurta')


# Connect to google sheet

In [2]:
# https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html

In [6]:
row = ["I'm","inserting","a","row","into","a,","Spreadsheet","with","Python"]
# index = 1
# sheet.insert_row(row, index)

import pygsheets

client = pygsheets.authorize(service_file='test-bq-rc-demo-a56264e3ae52.json')

# Open the spreadsheet and the first sheet.
sh = client.open('ScrapeDataSheet').sheet1

# Update a single cell.
sh.get_all_records()


[]